In [19]:
import numpy as np

from bokeh.layouts import row, column
from bokeh.models import CustomJS, Slider
from bokeh.plotting import figure, output_file, show, ColumnDataSource

def cost_curve(x,premium,contribution,deductible,coinsuranceRate,maxOutOfPocket):
    eff_premium = premium - contribution
    post_deduct = (coinsuranceRate * (x - deductible)) + deductible
    y = np.fmin(np.fmin(x, post_deduct), maxOutOfPocket) + eff_premium
    return y

In [25]:
x = np.linspace(0.0, 10000, 100)

premium_slider = Slider(start=0, end=2000, value=100, step=.1, title="Premium")
contribution_slider = Slider(start=0, end=1000, value=50, step=10, title="Company Contribution")
deductible_slider = Slider(start=0, end=6000, value=2000, step=10, title="Deductible")
coinsuranceRate_slider = Slider(start=0, end=1, value=0.5, step=.01, title="Coinsurance Rate (paid by you)")
maxOutOfPocket_slider = Slider(start=0, end=8000, value=4000, step=10, title="Maximum Out Of Pocket")
y = cost_curve(x,premium_slider.value,contribution_slider.value,deductible_slider.value,coinsuranceRate_slider.value,maxOutOfPocket_slider.value)

plot = figure(x_range=(0,10000), y_range=(0,10000), plot_width=400, plot_height=400, tools="pan,hover,crosshair,reset")
plot.line(x, x, line_dash=(4,4), line_color="black")
plot.xaxis.axis_label = 'Out-of-Pocket Expenses'
plot.yaxis.axis_label = 'Actual Incurred Cost'

source = ColumnDataSource(data=dict(x=x, y=y))
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6, legend="Plan 1")

callback = CustomJS(args=dict(source=source, premium=premium_slider, contribution=contribution_slider, deductible=deductible_slider, coinsuranceRate=coinsuranceRate_slider, maxOutOfPocket=maxOutOfPocket_slider),
                    code="""
    const data = source.data;
    const P = premium.value - contribution.value;
    const D = deductible.value;
    const R = coinsuranceRate.value;
    const M = maxOutOfPocket.value;
    const x = data['x']
    const y = data['y']
    for (var i = 0; i < x.length; i++) {
        const post_deduct = (R*(x[i] - D)) + D;
        y[i] = Math.min(Math.min(x[i],post_deduct),M) + P;
    }
    source.change.emit();
""")

premium_slider.js_on_change('value', callback)
contribution_slider.js_on_change('value', callback)
deductible_slider.js_on_change('value', callback)
coinsuranceRate_slider.js_on_change('value', callback)
maxOutOfPocket_slider.js_on_change('value', callback)

layout = row(
    plot,
    column(premium_slider, contribution_slider, deductible_slider, coinsuranceRate_slider, maxOutOfPocket_slider),
)

output_file("main.html", title="Insurance Plan Comparison")

show(layout)